### Interactive Zone-Based Object Counting with DeGirum PySDK
This script demonstrates how to perform object detection and counting within specific zones of interest using DeGirum PySDK. The key features include:

- Model-Based Object Detection: Utilizes the YOLOv8 model to detect objects such as cars, motorbikes, and trucks from a video stream.
- Zone-Based Counting: Counts objects within user-defined polygonal zones, defined as a list of coordinates.
- Interactive Zone Adjustment: The zones of interest can be interactively adjusted in real-time using the mouse within the display window, providing flexibility to adapt to changing scenarios.
- Customizable Output: Supports filtering specific object classes for counting and displays results per class.
- Stream Processing: Processes video streams, displaying both the detected objects and zone-based analytics in a dedicated window.

Simply define your zones, select the classes to track, and specify the video source to get started. This script is ideal for applications such as traffic monitoring, crowd analysis, and zone-specific surveillance.

In [ ]:
import degirum as dg, degirum_tools


inference_host_address = "@local"
zoo_url = "degirum/hailo"
token = ''
device_type = "HAILORT/HAILO8L"

# set the model name and video source
model_name = 'yolov8n_relu6_coco--640x640_quant_hailort_hailo8l_1'
video_source = '../assets/Traffic.mp4'

# define the zones of interest
polygon_zones = [
    [[265, 260], [730, 260], [870, 450], [120, 450]],
    [[400, 100], [610, 100], [690, 200], [320, 200]],
]

# define class list and display options
class_list = ["car", "motorbike", "truck"]
per_class_display = True
window_name="AI Camera"

# load model
model = dg.load_model(
    model_name=model_name, 
    inference_host_address=inference_host_address,
    zoo_url=zoo_url,
    token=token,
    overlay_color=[(255,0,0)],
    output_class_set = set(class_list)
)

# create zone counter
zone_counter = degirum_tools.ZoneCounter(
        polygon_zones,        
        class_list=class_list,
        per_class_display=per_class_display,
        triggering_position=degirum_tools.AnchorPoint.CENTER,
        window_name=window_name,  # attach display window for interactive zone adjustment
)

# attach zone counter to model
degirum_tools.attach_analyzers(model, [zone_counter])

# run inference and display results
with degirum_tools.Display(window_name) as display:
    for inference_result in degirum_tools.predict_stream(model, video_source,):
        display.show(inference_result)